In [ ]:
from dolfin import *
import numpy as np
import matplotlib.pyplot as plt
#%config InlineBackend.figure_format = 'retina'

In [ ]:
def errors(n, r=1):
    
    mesh = UnitSquareMesh(n, n)
    
    # start by solving for u using non-hybrid method
    S_element = FiniteElement('CG', mesh.ufl_cell(), r+1)
    V_element = FiniteElement('N2curl', mesh.ufl_cell(), r)
    W = FunctionSpace(mesh, S_element*V_element)
    
    bcs = [DirichletBC(W.sub(0), Constant(0), DomainBoundary()),
           DirichletBC(W.sub(1), Constant((0,0)), DomainBoundary())]

    s, u = TrialFunctions(W)
    t, v = TestFunctions(W) 
    a = (-inner(s, t) + inner(u, grad(t)) + inner(grad(s), v) + inner(curl(u), curl(v)))*dx
    f = Expression(('-2*pow(pi,2)*cos(pi*x[0])*sin(pi*x[1])', '2*pow(pi,2)*sin(pi*x[0])*cos(pi*x[1])'), degree=r+1)
    L = inner(f, v)*dx
    
    w = Function(W)
    solve(a == L, w, bcs)
    u = w.sub(1)
    
    curl_u = project(curl(u), FunctionSpace(mesh, "DG", r-1))
    
    # post-process
    V_dg_element = VectorElement('DG', mesh.ufl_cell(), r)
    Q_element = FiniteElement('CG', mesh.ufl_cell(), r+2)
    W = FunctionSpace(mesh, V_dg_element*Q_element)
    
    lam, q = TrialFunctions(W)
    v, p = TestFunctions(W)
    a = (inner(q, p) + inner(curl(q), curl(p)) - curl(lam*p + v*q) + 1e-10*inner(lam, v))*dx
    L = (inner(f, v + curl(p)) + inner(curl_u, p - curl(v)))*dx
    
    w = Function(W)
    solve(a == L, w)
    q = w.sub(1)

    # exact solutions
    u_exact = Expression(('-cos(pi*x[0])*sin(pi*x[1])', 'sin(pi*x[0])*cos(pi*x[1])'), degree=r+3)
    curl_u_exact = Expression('2*pi*cos(pi*x[0])*cos(pi*x[1])', degree=r+5)
    
    u_error = errornorm(u_exact, u)
    curl_u_error = errornorm(curl_u_exact, curl_u)
    q_error = errornorm(curl_u_exact, q)

    return u_error, curl_u_error, q_error

In [ ]:
ns = [2**k for k in range(1,5)]
es = [errors(n, r=3) for n in ns]

In [ ]:
plt.loglog(1/np.array(ns), es, basex=2, basey=2)
plt.legend(['$u$',r'$\operatorname{curl} u$','$q$'])
plt.xlabel('$h$')
plt.ylabel('error')

In [ ]:
es

In [ ]:
rates = -np.diff(np.log2(es), axis=0)
rates